## Make multiwavelength light curves using archival data

### Summary:
 - model plots after van Velzen et al. 2021, https://arxiv.org/pdf/2111.09391.pdf
 
### Input:
 - a catalog of CLAGN from the literature

### Output:
 - an archival optical + IR + neutrino light curve
 
### Technical Goals:
 - should be able to run from a clean checkout from github
 - should be able to automatically download all catalogs & images used
 - need to have all photometry in the same physical unit
 
### Authors:
IPAC SP team

### Acknowledgements:
Suvi Gezari, Antara Basu-zych,
MAST, HEASARC, & IRSA Fornax teams

In [19]:
from astroquery.ipac.ned import Ned
from astropy.coordinates import SkyCoord
from astroquery.heasarc import Heasarc


In [3]:
# use the following paper to make a sample of CLAGN: https://iopscience.iop.org/article/10.3847/1538-4357/aaca3a 

# This sample can later be switched out to a differen/larger sample of "interesting" targets

#use ADS to find the refcode for this paper
CLAGN = Ned.query_refcode('2018ApJ...862..109Y')


In [34]:
#do I want to keep these as an astropy table or convert them into something else?
# try to stay with astropy so it can keep the units once we start adding photometry
type(CLAGN)
CLAGN

No.,Object Name,RA,DEC,Type,Velocity,Redshift,Redshift Flag,Magnitude and Filter,Separation,References,Notes,Photometry Points,Positions,Redshift Points,Diameter Points,Associations
,,degrees,degrees,,km / s,,,,arcmin,,,,,,,
int32,str30,float64,float64,object,float64,float64,object,object,float64,int32,int32,int32,int32,int32,int32,int32
1,WISEA J000107.52-000552.5,0.28136,-0.09789,G,59147.0,0.197292,SPEC,19.7g,--,6,0,34,8,5,4,0
2,WISEA J012648.10-083948.0,21.70037,-8.66335,G,59333.0,0.197913,SPEC,18.6g,--,22,0,62,14,8,8,0
3,2MASS J01595763+0033105,29.99,0.55301,QSO,93546.0,0.312036,SPEC,19.6g,--,63,0,76,17,19,10,0
4,WISEA J015957.63+003310.3,29.99015,0.55288,IrS,93547.0,0.31204,SLS,,--,4,0,12,3,2,0,0
5,WISEA J080347.56+425839.0,120.94815,42.97747,G,125547.0,0.418778,SPEC,21.8g,--,4,0,27,5,3,4,0
6,WISEA J083132.25+364617.0,127.88438,36.77146,G,58463.0,0.19501,SPEC,18.8g,--,17,0,62,9,7,10,0
7,WISEA J084957.78+274728.8,132.49077,27.79139,QSO,89499.0,0.298537,SPEC,19.1g,--,23,0,65,12,11,8,0
8,WISEA J090932.02+474730.7,137.38346,47.79186,G,35059.0,0.116944,SPEC,18.7g,--,19,0,53,8,6,8,1


In [18]:
# Build a list of skycoords from target ra and dec
coords_list = [
    SkyCoord(ra, dec, frame='icrs', unit='deg')
    for ra, dec in zip(CLAGN['RA'], CLAGN['DEC'])
]


### Find photometry for these targets in NASA catalogs
- look at NAVO use cases to get help with tools to do this
- deciding up front to use astroquery instead of pyvo
    - astroquery is apparently more user friendly


In [30]:
#list all the available HEASARC missions
heasarc = Heasarc()
mission_table = heasarc.query_mission_list()
mission_table.pprint_all()

#now what, there are a thousand of these.  I can guess, or we can loop over all of them??
#guess at 'chanmaster' for now


    Mission         Table                                    Table Description                                
---------------- ------------ --------------------------------------------------------------------------------
           AGILE     agilecat                         First AGILE Catalog of High-Confidence Gamma-Ray Sources
           AGILE   agileupvar                Updated AGILE Catalog of Bright Gamma-Ray Sources and Variability
           AKARI     akaribsc                        AKARI/FIS All-Sky Survey Bright Source Catalog, Version 1
           AKARI     akaripsc                         AKARI/IRC All-Sky Survey Point Source Catalog, Version 1
             ANS   ansuvpscat                              ANS Ultraviolet Photometry Catalog of Point Sources
         ARIEL V      ariel3a                                                          3rd Ariel-V SSI Catalog
         ARIEL V       ariel5                                                          Ariel V All-Sky Monitor
 

In [33]:
#figure out what the column names are in chanmaster
cols = heasarc.query_mission_cols(mission='fermi3fgl')
cols
#ok, chanmaster is not a photometry catalog.
#it seems pretty futile and reuqires human input for me to just be guessing at the correct catalos to use
#maybe I should loop over all of them.
#how long would that take anyway?
#but they all have different column names for reporting the fluxes, so that is also pretty useless


['NAME',
 'RA',
 'DEC',
 'FLUX_1_100_GEV',
 'FLUX_1_100_GEV_ERROR',
 'SPECTRAL_INDEX',
 'SPECTRAL_INDEX_ERROR',
 'DETECTION_SIGNIFICANCE',
 'ASSOC_NAME_1',
 'ALT_GAMMARAY_NAME_1',
 'ALT_GAMMARAY_NAME_2',
 'ALT_GAMMARAY_NAME_3',
 'ALT_GAMMARAY_NAME_4',
 'ALT_GAMMARAY_NAME_5',
 'ALT_GAMMARAY_NAME_6',
 'ALT_GAMMARAY_NAME_7',
 'ANALYSIS_FLAGS',
 'ASSOC_NAME_2',
 'ASSOC_NAME_3',
 'BETA',
 'BETA_ERROR',
 'BII',
 'CURVE_SIGNIFICANCE',
 'CUTOFF',
 'CUTOFF_ERROR',
 'ENERGY_FLUX',
 'ENERGY_FLUX_ERROR',
 'EXP_INDEX',
 'EXP_INDEX_ERROR',
 'FLUX_0P3_1_GEV',
 'FLUX_0P3_1_GEV_NEG_ERR',
 'FLUX_0P3_1_GEV_POS_ERR',
 'FLUX_100_300_MEV',
 'FLUX_100_300_MEV_NEG_ERR',
 'FLUX_100_300_MEV_POS_ERR',
 'FLUX_10_100_GEV',
 'FLUX_10_100_GEV_NEG_ERR',
 'FLUX_10_100_GEV_POS_ERR',
 'FLUX_1_3_GEV',
 'FLUX_1_3_GEV_NEG_ERR',
 'FLUX_1_3_GEV_POS_ERR',
 'FLUX_30_100_MEV',
 'FLUX_30_100_MEV_NEG_ERR',
 'FLUX_30_100_MEV_POS_ERR',
 'FLUX_3_10_GEV',
 'FLUX_3_10_GEV_NEG_ERR',
 'FLUX_3_10_GEV_POS_ERR',
 'FLUX_DENSITY',
 'FLUX_DEN

### Current Questions to start working on this project:
 - What is the best way to find all available photometry across all NASA archives?
     - could write a loop over all catalogs for all archives, but they all have different flux column names
 - if that is prohibitive, how do I choose which catalogs to search - human input?, and which human?
 - each catalog has a different name for the flux column, how do I automatedly add that flux column to my astropy table?
 - astropy.ned useful? or work with the individual archives? or which specific tools? 
 - need to define when a match is good enough to include photometry in our light curve.

In [5]:

#astroquery Heasarc query_region can take coords and radius as input
#1.it requires mission as an input, 
#2. definitely check Fermi - gamma ray observatory


#For all CLAGN coords in the paper
for c = range(1):
    #do a query on position
        #do I need to define which catalogs(mission) I want to search up front?
    mission = 'fermi3fgl'
    radius = 0.1*u.degree
    results = heasarc.query_region(coords(c), mission = mission, radius = radius, sortvar = 'SEARCH_OFFSET_')
    #if there is a good match where good = ??
        #save the found photometry in the astropy table
        #do i need to make columns for this up front?

In [6]:
#astroquery.ipac.irsa requires a catalog input s
#1. can it acccess ZTF and GAIA?


In [7]:
#astroquery MAST doesn't require a catalog input but we might want it to narrow things down?
#1. Pan-STARRS
#2. MAST has copies of ATLAS all-sky stellar reference catalog- but not searchable
    #- might be available through astroquery.vizier
    


### Find photometry for these targets in relevant, non-NASA catalogs


In [8]:
#ASAS-SN (all sky automated survey for supernovae) has a website that can be manually searched
# - see if astroquery.vizier can find it



In [9]:
#icecube has a 2008 - 2018 catalog which we can download and is small - https://icecube.wisc.edu/data-releases/2021/01/all-sky-point-source-icecube-data-years-2008-2018/

### Find those targets in big data WISE light curve catalog that Dave Shupe built


In [10]:
#How many of the CLAGN targets are covered by this catalog?  enough to do this work? maybe > 5?
#1. Use existing code in irsa SP that works with this catalog
#2. figure out how to efficiently search that catalog since it can't be loaded into memory
    #1. somehow I think this can be done in a cross match early on, so I only return rows that match
    #2. test DASK, VAEX, AXS, ???
#3. do we need updates to this catalog from Dave?

### Make plots of luminosity as a function of time
- time could be days since peak, or days since first observation, or??

### ML Extension 
Consider training a ML model to do light curve classification based on this sample of CLAGN
 - once we figure out which bands these are likely to be observed in, could then have a optical + IR light curve classifier
 - what would the features of the light curve be?
 - what models are reasonable to test as light curve classifiers?
 - could we make also a sample of TDEs, SNe, flaring AGN? - then train the model to distinguish between these things?
 - need a sample of non-flaring light curves
 
After training the model:
 - would then need a sample of optical + IR light curves for "all" galaxies = big data to run the model on.
